# Star Formation

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import savgol_filter
from multiprocessing import Pool
import os

/u/fiza/conda-envs/auriga/lib/python3.7/site-packages/setuptools/_importlib.py:23: UserWarning: `importlib-metadata` version is incompatible with `setuptools`.
This problem is likely to be solved by installing an updated version of `importlib-metadata`.
  warnings.warn(msg)  # Ensure a descriptive message is shown.


In [2]:
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
import numpy as np

In [3]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup
from auriga.settings import Settings
from auriga.support import make_snapshot_number
from auriga.paths import Paths
from auriga.parser import parse

In [4]:
figure_setup()
settings = Settings()

In [5]:
DISC_STD_CIRC = settings.disc_std_circ
DISC_MIN_CIRC = settings.disc_min_circ
COLD_DISC_DELTA_CIRC = settings.cold_disc_delta_circ
BULGE_MAX_SPECIFIC_ENERGY = -0.6
SUFFIX = "_02"

In [6]:
SAMPLE = [f"au{i}_or_l4" for i in settings.groups["Included"]]

In [7]:
def calculate_sfr_by_region(simulation: str) -> np.ndarray:
    _, _, _, snapshot = parse(simulation)
    settings = Settings()

    if snapshot >= 40:  # Here, settings.first_snap doesn't work
        s = Snapshot(simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
        s.tag_particles_by_region(
            disc_std_circ=DISC_STD_CIRC,
            disc_min_circ=DISC_MIN_CIRC,
            cold_disc_delta_circ=COLD_DISC_DELTA_CIRC,
            bulge_max_specific_energy=BULGE_MAX_SPECIFIC_ENERGY)
        sfr = s.calculate_sfr_by_region()
    else:
        sfr = [np.nan] * 4

    return np.array(sfr)

In [8]:
def calculate_sfr_by_region_evolution(simulation: str) -> pd.DataFrame:
    galaxy, rerun, resolution = parse(simulation)
    n_snapshots = make_snapshot_number(rerun, resolution)
    snapnums = [f"{simulation}_s{i}" for i in range(n_snapshots)]
    sfr = np.array(Pool().map(calculate_sfr_by_region, snapnums))

    # Read time
    paths = Paths(galaxy, rerun, resolution)
    time = pd.read_csv(
        f"../{paths.results}temporal_data.csv", usecols=["Time_Gyr"])

    # Create dataframe
    data = {"SFR_Msun/yr_H": sfr[:, 0], 
            "SFR_Msun/yr_B": sfr[:, 1], 
            "SFR_Msun/yr_CD": sfr[:, 2], 
            "SFR_Msun/yr_WD": sfr[:, 3],
            "SFR_Msun/yr": np.sum(sfr, axis=1),
            "Time_Gyr": time["Time_Gyr"].to_numpy()}

    return pd.DataFrame(data)

In [11]:
fig = plt.figure(figsize=(7, 8))
gs = fig.add_gridspec(nrows=6, ncols=4, hspace=0.0, wspace=0.0)
axs = gs.subplots(sharex=True, sharey=True)

for ax in axs.flatten():
    ax.tick_params(which='both', direction="in")
    if ax == axs[-1, -1]: ax.axis("off")
    ax.set_xlim(0, 14)
    ax.set_ylim(0.1, 20),
    ax.set_xticks([2, 4, 6, 8, 10, 12])
    ax.set_yscale("log")
    ax.set_yticks([0.1, 1, 10])
    ax.set_yticklabels([0.1, 1, 10])
    if ax.get_subplotspec().is_last_row() or ax == axs[-2, -1]:
        ax.set_xlabel("Time [Gyr]")
        ax.tick_params(labelbottom=True)
    if ax.get_subplotspec().is_first_col():
        ax.set_ylabel(r"SFR [$\mathrm{M}_\odot \, \mathrm{yr}^{-1}$]")

for i, simulation in enumerate(SAMPLE):
    ax = axs.flatten()[i]
    label = f"Au{parse(simulation)[0]}"
    sfr = calculate_sfr_by_region_evolution(simulation)
    sfr.dropna(inplace=True)
    for component in settings.components:
        ax.plot(sfr["Time_Gyr"],
                savgol_filter(sfr[f"SFR_Msun/yr_{component}"], 5, 1),
                c=settings.component_colors[component],
                lw=1.0, label=settings.component_labels[component], zorder=10)
    ax.plot(sfr["Time_Gyr"], savgol_filter(sfr["SFR_Msun/yr"], 5, 1),
            c='k', lw=1.0, label="Total", zorder=10)
    ax.text(x=0.95, y=0.95, size=6.0,
            s=r"$\texttt{" + label + "}$",
            ha='right', va='top', transform=ax.transAxes)

    if ax == axs[0, 0]:
        ax.legend(loc="lower center", framealpha=0.0, fontsize=5.0,
                  bbox_to_anchor=(3.5, -4.9), ncol=1)

    ax.xaxis.label.set_size(8.0)
    ax.yaxis.label.set_size(8.0)

    fig.savefig(f"../images/sfr_by_region/included{SUFFIX}.pdf")

plt.close(fig)